In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import Dataset, DataLoader

### need visdom server

In [2]:
# import visdom

# vls = visdom.Visdom()
# vls.close(env = "main")

# def loss_tracker(loss_plot, loss_value, num):
#     vls.line(X = num, Y = loss_value, win = loss_plot, update = "append")

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(777)
if device == "cuda":
    torch.cuda.manual_seed_all(777)

print(device)

cuda


In [4]:
# parameters
learning_rate = 0.001
epochs = 15
batch_size = 100

In [5]:
#MNIST dataset

mnist_train = dsets.MNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root="MNIST_data/", train=False, transform=transforms.ToTensor(), download=True)

In [6]:
# data loader
data_loader = DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle = True, drop_last = True)

In [7]:
# model create
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        relu = torch.nn.ReLU()
        maxpool = torch.nn.MaxPool2d(2)
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1), relu, maxpool)
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1), relu, maxpool)
        # view를 사용해 conv layer의 결과를 fc통과를 위해 1차원을 펴줘야한다. (batch_size, -1)
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [8]:
model = CNN().to(device)

In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [10]:
# need visdom server on
# loss_plt = vls.line(Y = torch.Tensor(1).zero_(), opts = dict(title = "loss_tracker", legend = ["loss"], showlegend = True))

In [11]:
# training

total_batch = len(data_loader)

for epoch in range(epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        predict = model(X)
        cost = criterion(predict, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print(f"Epoch: {epoch}/{epochs} cost: {avg_cost} ")
     
    # need visdom server on
    # loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))

Epoch: 0/15 cost: 0.2203698754310608 
Epoch: 1/15 cost: 0.06091485172510147 
Epoch: 2/15 cost: 0.046026792377233505 
Epoch: 3/15 cost: 0.03662533313035965 
Epoch: 4/15 cost: 0.029923712834715843 
Epoch: 5/15 cost: 0.026077285408973694 
Epoch: 6/15 cost: 0.020586222410202026 
Epoch: 7/15 cost: 0.018512621521949768 
Epoch: 8/15 cost: 0.015310007147490978 
Epoch: 9/15 cost: 0.012990032322704792 
Epoch: 10/15 cost: 0.011465680785477161 
Epoch: 11/15 cost: 0.008422893472015858 
Epoch: 12/15 cost: 0.00866619125008583 
Epoch: 13/15 cost: 0.006366122514009476 
Epoch: 14/15 cost: 0.006272963248193264 


In [12]:
test_loader = DataLoader(dataset = mnist_test, batch_size = batch_size, shuffle = False)

with torch.no_grad():
    # cuda 메모리 부족..
    #X = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    #Y = mnist_test.test_labels.to(device)
    sum = 0
    total_batch = len(test_loader)
    for X, Y in test_loader:
        X = X.to(device)
        Y = Y.to(device)
        predict = model(X)
        correct = torch.argmax(predict, 1) == Y
        sum += correct.float().mean()
    accuracy = sum / total_batch
    print(f"Accuracy: {accuracy}")

Accuracy: 0.9893001317977905
